# 1. Custom Data Preparation

## 1-1. 데이터 병합
* '인도보행 영상' 데이터가 담긴 (1) 여러 개의 zip 폴더를 압축 해제한 후,
* (2) 모든 파일들을 하나의 새로운 폴더('dataset')로 복사하여 이동시킴

In [ ]:
import os
import shutil

In [ ]:
base_path = '/content/drive/MyDrive/darknet/data/dataset'
file_names = os.listdir(base_path)
# file_names.remove('dataset')
len(file_names)               # 파일 17850 개

In [ ]:
all_files = []
for fi in file_names:
  gag_file = base_path+'/'+fi #/content/drive/MyDrive/data/sumfile/Bbox~
  li = os.listdir(gag_file) # '*.jpg'
  print(li)
  for l in li:
    all_files.append(gag_file+'/'+l)

for mv in all_files:
    try:
        shutil.move(mv, '/content/drive/MyDrive/data/sumfile/dataset')
    except shutil.Error as e:
        print(f"Error: {e}")

In [ ]:
len(os.listdir('/content/drive/MyDrive/data/sumfile/dataset'))  # 파일 17850 개 모두 복사 확인

## 1-2. 객체 Annotation 및 Indexing
* 원데이터의 기존 클래스를 새로운 기준으로 재분류 및 인덱싱

In [ ]:
label_to_index =   {
    'green_light' : 0,
    'red_light' : 1,
    'carrier' : 2, 'stroller' : 2, 'wheelchair' : 2,
    'motorcycle' : 3, 'scooter' : 3,
    'person' : 4,
    'bench' : 5, 'chair' : 5, 'table' : 5,
    'stop' : 6,
    'kiosk' : 7,
    'movable_signage' : 8, 'traffic_light_controller' : 8,
    'barricade' : 8, 'pole' : 8, 'parking_meter' : 8,
    'bollard' : 8, 'fire_hydrant' : 8, 'tree_trunk' : 8,
    'potted_plant' : 8, 'power_controller' : 8,
    'bicycle' : 9,
    'bus' : 10, 'car' : 10, 'truck' : 10,
    'cat' : 11, 'dog' : 11, 'traffic_light' : 11, 'traffic_sign' : 11,  # 11번 라벨은 우리 프로젝트 타겟 유저에게 중요성(위험도)이 크지 않다고 생각되어 삭제
    }

In [ ]:
index_to_classname = {
    'Green' : 0,
    'Red' : 1,
    'Mobility Aid' : 2,
    'Motorcycle' : 3,
    'Person' : 4,
    'Rest' : 5,
    'Stop' : 6,
    'Kiosk' : 7,
    'Obstacle' : 8,
    'Bike' : 9,
    'Car' : 10,
    }

## 1-3. XML 파일을 TXT 파일로

In [ ]:
import os
import glob
import pandas as pd

import xml.etree.ElementTree as ET
import xml.dom.minidom
from tqdm import tqdm

In [ ]:
%cd /content/drive/MyDrive/darknet/data/dataset

In [ ]:
filepath = '/content/drive/MyDrive/darknet/data/dataset/'

In [ ]:
xml_list = []
for i in glob.glob('*.xml') :
    xml_list.append(i)

In [ ]:
label_to_index =   {
    'green' : 0,
    'red' : 1,
    'carrier' : 2, 'stroller' : 2, 'wheelchair' : 2, 'motorcycle' : 2, 'scooter' : 2, 'bicycle' : 2,
    'person' : 3,
    'bench' : 4, 'chair' : 4, 'table' : 4,
    'stop' : 5,
    'bollard' : 6,
    'fire_hydrant' : 7, 'power_controller' : 7, 'parking_meter' : 7, 'movable_signage' : 7, 'barricade' : 7,
    'potted_plant' : 8, 'tree_trunk' : 8,
    'bus' : 9, 'car' : 9, 'truck' : 9,
    'pole' : 10,
    'cat' : 11, 'dog' : 11, 'traffic_light' : 11, 'traffic_sign' : 11, 'traffic_light_controller' : 11, 'kiosk' : 11
    }

In [ ]:
index_to_classname = {
    'Green_light' : 0,
    'Red_light' : 1,
    'Vehicle' : 2,
    'Person' : 3,
    'Rest' : 4,
    'Stop' : 5,
    'Bollalrd' : 6,
    'Obstacle' : 7,
    'Plant' : 8,
    'Car' : 9,
    'Pole' : 10,
    'Others' : 11
    }

In [ ]:
exclude_label = [11]

In [ ]:
for xml_file in tqdm(xml_list) :

  tree = ET.parse(xml_file)
  root = tree.getroot()
  image_data = root.findall('image')

  for data in image_data :
      image_height = data.items()[1][1] # 이미지들의 크기, 파일 이름을 가져온다
      image_width = data.items()[2][1]
      image_name = data.items()[3]

      object_data = ''
      for image_info in data.findall('box') : #이미지에 있는 박스 하나에 대한 데이터를 가져온다

          info = image_info.items()

          label = info[5][1]
          label = label_to_index[label]

          if label in exclude_label : # 라벨이 11번일 경우 기록하지 않고 넘어간다.
            continue

          occluded = float(info[0][1]) # 물체 가려짐 유무
          xbr = float(info[2][1])
          ybr = float(info[3][1])
          xtl = float(info[4][1])
          ytl = float(info[1][1])


          txt_x = ((xbr + xtl) / 2) / int(image_width)
          txt_y = ((ybr + ytl) / 2) / int(image_height)
          txt_w = (xbr - xtl) / int(image_width)
          txt_h = (ytl - ybr) / int(image_height)

          img_size = 416 # resize할 이미지 크기 고려
          img_w = 1920
          img_h = 1080
          ratio = img_size / 1920

          img_w *= ratio
          img_h *= ratio

          # dw, dh : padding 되는 영역
          dw = (img_size-img_w)/2
          dh = (img_size-img_h)/2

          # padding 영역을 비율로 바꿈
          w_pad_ratio = dw / img_size
          h_pad_ratio = dh / img_size

          x = (float(txt_x) * img_w / img_size) + w_pad_ratio
          y = (float(txt_y) * img_h / img_size) + h_pad_ratio

          w_adjusted = float(txt_w) * img_w / img_size
          h_adjusted = float(txt_h) * img_h / img_size

          object_data += f'{label} {x} {y} {w_adjusted} {h_adjusted}\n'

      with open(filepath + image_name[1][:-4] + '.txt', 'wt') as f:
          f.write(object_data[:-1])

## 2-1. 이미지 리사이징 : 416x416
* YOLO 모델은 32만큼 downsampling 하기 때문에, 보통 입력 이미지의 해상도 32의 배수가 되도록 함 (예: v4 416, v5 640)

In [ ]:
for filename in tqdm(txt_data) :

  # img_path 순회하면서 코드 반복
  txt_file = filename + '.txt'

  path = img_path+'/'+filename
  img = cv2.imread(path)
  img_size = 416
  if(img.shape[1] > img.shape[0]) :
      ratio = img_size/img.shape[1]
  else :
      ratio = img_size/img.shape[0]

  img = cv2.resize(img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

  w, h = img.shape[1], img.shape[0]
  w = 1920 # 데이터의 크기가 1920*1080으로 고정되어 있다
  h = 1080
  ratio = img_size / 1920
  w *= ratio
  h *= ratio

  dw = (img_size-w)/2
  dh = (img_size-h)/2

  M = np.float32([[1,0,dw], [0,1,dh]])
  img_re = cv2.warpAffine(img, M, (img_size, img_size), borderValue=(0, 0, 0))

  save_path = '/content/drive/MyDrive/darknet/data/resize/'+ filename

  cv2.imwrite(save_path , img_re)

  txt_path = '/content/drive/MyDrive/darknet/data/resize/'+ txt_file


  w_pad_ratio = dw / img_size
  h_pad_ratio = dh / img_size

  # txt파일 resize한 크기에 맞게 변경
  with open(img_path+'/'+txt_file, 'r') as f:
    lines = f.read().split('\n')

  object_data = ''
  for line in lines:
    try :
      label, txt_x, txt_y, txt_w, txt_h = line.split(' ')
    except :
      continue
    x = (float(txt_x) * w / img_size) + w_pad_ratio
    y = (float(txt_y) * h / img_size) + h_pad_ratio

    w_adjusted = float(txt_w) * w / img_size
    h_adjusted = float(txt_h) * h / img_size

    object_data += f'{label} {x} {y} {w_adjusted} {h_adjusted}\n'

  with open(txt_path, 'wt') as f:
    f.write(object_data[:-1])

# 3. 모델링1: YOLO v4
* 참고: 02_1(solution)YOLOv4_train with Darknet.ipynb
* https://dohyeon.tistory.com/9

In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

## 3-1. 클로닝

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15833, done.
remote: Total 15833 (delta 0), reused 0 (delta 0), pack-reused 15833
Receiving objects: 100% (15833/15833), 14.39 MiB | 13.54 MiB/s, done.
Resolving deltas: 100% (10666/10666), done.


## 3-2. Custom 데이터 로딩 (from Gdrive)

In [ ]:
## 데이터 경로 저장
file_path = '/content/drive/MyDrive/darknet/data/datatest1'

file_name_list = os.listdir(file_path)

file_names = {}

for fnl in file_name_list:
  if fnl[:-4] not in file_names:
    file_names[fnl[:-4]] = 1
  else:
    file_names[fnl[:-4]] += 1

file_names.items()

In [ ]:
data_list = []
for fndk,fndv in file_names.items():
  if fndv == 2:
    data_list.append(os.path.join(file_path, fndk+'.png'))

data_list[:4]

## 3-3. 파일 생성

In [ ]:
## train.txt, valid.txt, test.txt, ClassNames.names 생성 // 6:2:2
X_file, X_test= train_test_split(data_list, test_size=0.2, random_state=777)

X_train, X_val= train_test_split(X_file, test_size=0.2, random_state=777)

### 1) .data파일 생성
* train.txt, valid.txt, test.txt
* ClassNames.names
* mask_data.data

*아래 코드 플젝에 맞게 수정*

In [ ]:
f1 = open('/content/drive/MyDrive/darknet/data/train2.txt', 'w')
for tr in X_train:
  f1.write(tr+'\n')
f1.close()

f2 = open('/content/drive/MyDrive/darknet/data/val2.txt', 'w')
for v in X_val:
  f2.write(v+'\n')
f2.close()

f3 = open('/content/drive/MyDrive/darknet/data/test2.txt', 'w')
for ts in X_test:
  f3.write(ts+'\n')
f3.close()

In [ ]:
f4 = open('/content/drive/MyDrive/darknet/data/ClassNames.names', 'w')
f4.write('''Green_light
Red_light
Mobility Aid
Motorcycle
Person
Bench
Stop
Kiosk
Obstacle
Bicycle
Car''')
f4.close()

In [ ]:
## find_obstacle_data.data 생성
f5 = open('/content/drive/MyDrive/darknet/find_obstacle_data.data', 'w')
f5.write('''classes = 11
train = /content/drive/MyDrive/darknet/data/train2.txt
valid = /content/drive/MyDrive/darknet/data/val2.txt
test = /content/drive/MyDrive/darknet/data/test2.txt
names = /content/drive/MyDrive/darknet/data/ClassNames.names
backup = backup''')
f5.close()

### 2) cfg 폴더 내 'yolo4-custom.cfg' 파일 수정

* subdivision = 16
* width & height = 416
* max_batches = 22000 = num_classes * 2000 = 11 * 2000
* steps = 17600, 19800 = max_batches * 0.8, max_batches * 0.9

* yolo layer 마다 n_classes = 11
* yolo layer 앞에 있는 Conv layer 마다 n_filters = 48 = (num_classes + 5)*3

In [ ]:
import shutil
shutil.move('/content/yolov4-custom.cfg','/content/drive/MyDrive/darknet/cfg/yolov4-custom.cfg')

In [ ]:
# 변경사항이 반영이 되었는지 확인
%cat /content/darknet/cfg/yolov4-custom.cfg

### 3) 사전학습된 weights 다운로드

In [ ]:
%cd '/content/darknet/'

In [ ]:
# yolov4 pre-trained weights 다운로드
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# /content/darknet/yolov4.conv.137

## 3-4. Darknet 구축
1) Makefile 수정하여 옵션 변경

In [ ]:
# GPU, OPENCV, LIBSO을 true로 설정
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
# 변경 내역 확인하기
%cat /content/darknet/Makefile

2) 변경 후 Makefile 컴파일

In [ ]:
# 컴파일하면 darknet.py 사용 가능
!make

## 3-5. 전이 학습: YOLO v4

In [ ]:
import subprocess

# 실행 권한 부여 명령어
command = "chmod +x /content/drive/MyDrive/darknet/darknet"

# subprocess를 사용하여 명령어 실행
subprocess.run(command, shell=True)

In [ ]:
!/content/drive/MyDrive/darknet/darknet detector train /content/drive/MyDrive/darknet/data/find_obstacle_data.data /content/drive/MyDrive/darknet/cfg/yolov4-custom.cfg /content/drive/MyDrive/darknet/yolov4.conv.137 -dont_show -map

In [ ]:
## 학습 재게
# %cd /content/darknet
# !./darknet detector train data/mask_data.data cfg/yolov4-custom.cfg backup/yolov4-custom_last.weights -dont_show -map

학습된 모델 사용해보기

In [ ]:
from darknet import *
network, class_names, class_colors = load_network("cfg/yolov4-custom.cfg", "data/find_obstacle_data.data", "backup/yolov4-custom_best.weights")

In [ ]:
width = network_width(network)
height = network_height(network)

print(width, height)

In [ ]:
from matplotlib import pyplot as plt
import cv2

# object detection을 수행하는 darknet_helper 함수를 살펴보자.
## input : img - inference image / width, height - network input size
## output : detections - detect result / width_ratio, height_ratio - img size ratio(inference img with net input size)
def darknet_helper(img, width, height):
#####################################################################################################################
  # 이미지를 darknet style로 전처리 -> darknet_image 변수에 저장
  darknet_image = make_image(width, height, 3)
  # input image를 BRG에서 RGB 순서로 변환 : cvtColor
  cv_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  # input image의 size를 416x416로 변환 (network에서 받아들이는 이미지의 크기가 416x416였기 때문에) : resize
  img_resized = cv2.resize(cv_img, (width, height), interpolation=cv2.INTER_LINEAR)
  # bounding box의 크기를 조정하기 위하여 이미지 크기 비율을 계산 # img.shape 활용
  img_height,img_width,_ = img.shape
  width_ratio = img_width/width # 원본 이미지 width/ network width
  height_ratio = img_height/height # 원본 이미지 height/ network height

  # darknet stlye로 전처리된 이미지로 모델을 작동시켜 detection을 수행
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  # detect_image 함수 실행
  detections = detect_image(network, class_names, darknet_image) #추론값
  free_image(darknet_image)

  return detections, width_ratio, height_ratio

In [ ]:
for label, confidence, bbox in detections:
  # center x, center y, width, height로 반환 받은 boundary box를 corner 정보 (left, top, right, bottom)로 변환
  left, top, right, bottom = bbox2points(bbox)
  #####################################################################################################################
  # 앞서 계산한 이미지 비율을 사용하여 boundary box의 크기를 조정
  left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
  # boundary box 좌표에 해당하는 사각형을 이미지 위에 그림 : cv2.rectangle
  cv2.rectangle(image, (left, top), (right, bottom), class_colors[label], 2)
  # 클래스 이름과 confidence level을 사각형 위에 입력 : cv2.putText
  text = f"{label}, {float(confidence):.2f}"
  text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
  text_x = left + (right - left) // 2 - text_size[0] // 2
  text_y = bottom + 15
  cv2.putText(image, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, class_colors[label], 2)

# boundary box를 추가한 이미지 출력
#RGB 이미지를 BGR로 변환
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2_imshow(image)

# 4. 모델링2: YOLO World

In [ ]:
!git clone --recursive https://github.com/AILab-CVC/YOLO-World
%cd YOLO-World/

In [ ]:
!pip install -U ultralytics
from ultralytics import YOLO

In [ ]:
# Initialize a YOLO-World model
model = YOLO('yolov8s-world.pt')  # or select yolov8m/l-world.pt

In [ ]:
# Define custom classes
model.set_classes(["bollard", "4 wheel vehicle", "2 wheel vehicle", "person", "green_traffic_light",  "red_traffic_light", "crosswalk", "obstacle"])

In [ ]:
# Save the model with the defined offline vocabulary
model.save("custom_yolov8s.pt")

In [ ]:
results = model.predict('/content/img4.jpg')
results[0].show()

# 5. 모델링3: YOLO v8

5-1 YOLO v8에 맞게 640*640으로 resize

In [ ]:
%cd /content/drive/MyDrive/darknet/data/dataset

In [ ]:
filepath = '/content/drive/MyDrive/darknet/data/dataset/'

In [ ]:
xml_list = []
for i in glob.glob('*.xml') :
    xml_list.append(i)

In [ ]:
label_to_index =   {
    'green' : 0,
    'red' : 1,
    'carrier' : 2, 'stroller' : 2, 'wheelchair' : 2, 'motorcycle' : 2, 'scooter' : 2, 'bicycle' : 2,
    'person' : 3,
    'bench' : 4, 'chair' : 4, 'table' : 4,
    'stop' : 5,
    'bollard' : 6,
    'fire_hydrant' : 7, 'power_controller' : 7, 'parking_meter' : 7, 'movable_signage' : 7, 'barricade' : 7,
    'potted_plant' : 8, 'tree_trunk' : 8,
    'bus' : 9, 'car' : 9, 'truck' : 9,
    'pole' : 10,
    'cat' : 11, 'dog' : 11, 'traffic_light' : 11, 'traffic_sign' : 11, 'traffic_light_controller' : 11, 'kiosk' : 11
    }

In [ ]:
exclude_label = [11]

In [ ]:
index_to_classname = {
    'Green_light' : 0,
    'Red_light' : 1,
    'Vehicle' : 2,
    'Person' : 3,
    'Rest' : 4,
    'Stop' : 5,
    'Bollalrd' : 6,
    'Obstacle' : 7,
    'Plant' : 8,
    'Car' : 9,
    'Pole' : 10,
    'Others' : 11
    }

In [ ]:
for xml_file in tqdm(xml_list) :

  tree = ET.parse(xml_file)
  root = tree.getroot()
  image_data = root.findall('image')

  for data in image_data :
      image_height = data.items()[1][1] # 이미지들의 크기, 파일 이름을 가져온다
      image_width = data.items()[2][1]
      image_name = data.items()[3]

      object_data = ''
      for image_info in data.findall('box') : #이미지에 있는 박스 하나에 대한 데이터를 가져온다

          info = image_info.items()

          label = info[5][1]
          label = label_to_index[label]

          if label in exclude_label : # 라벨이 11번일 경우 기록하지 않고 넘어간다.
            continue

          occluded = float(info[0][1]) # 물체 가려짐 유무
          xbr = float(info[2][1])
          ybr = float(info[3][1])
          xtl = float(info[4][1])
          ytl = float(info[1][1])


          txt_x = ((xbr + xtl) / 2) / int(image_width)
          txt_y = ((ybr + ytl) / 2) / int(image_height)
          txt_w = (xbr - xtl) / int(image_width)
          txt_h = (ytl - ybr) / int(image_height)

          img_size = 640
          img_w = 1920
          img_h = 1080
          ratio = img_size / 1920

          img_w *= ratio
          img_h *= ratio

          # dw, dh : padding 되는 영역
          dw = (img_size-img_w)/2
          dh = (img_size-img_h)/2

          # padding 영역을 비율로 바꿈
          w_pad_ratio = dw / img_size
          h_pad_ratio = dh / img_size

          x = (float(txt_x) * img_w / img_size) + w_pad_ratio
          y = (float(txt_y) * img_h / img_size) + h_pad_ratio

          w_adjusted = float(txt_w) * img_w / img_size
          h_adjusted = float(txt_h) * img_h / img_size

          object_data += f'{label} {x} {y} {w_adjusted} {h_adjusted}\n'

      with open(filepath + image_name[1][:-4] + '.txt', 'wt') as f:
          f.write(object_data[:-1])

5-2 데이터 증강을 위한 이미지 자르기

In [ ]:
import cv2
import os
import glob
import pandas as pd
import cv2
import imutils
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
%cd /content/drive/MyDrive/darknet/data/resize

In [ ]:
img_list = []
for i in glob.glob('*.png') :
    img_list.append(i[:-4])

In [ ]:
txt_list = []
for i in glob.glob('*.txt') :
    txt_list.append(i[:-4])

In [ ]:
augment_labels = ['2','4','5','7']

In [ ]:
for data_name in tqdm(img_list) :

  img_name = data_name + '.png'
  loaded_image = cv2.imread(img_name)
  text_data = data_name + '.txt'
  try :
    image_width = loaded_image.shape[0]
    image_height = loaded_image.shape[1]

    with open(text_data, 'r') as f:
      txt_lines = f.read().split('\n')
  except :
    continue
  for ind, single_line in enumerate(txt_lines) :
    if single_line and single_line.split()[0] in augment_labels :
      label = single_line.split()[0]
      x = float(single_line.split()[1])
      y = float(single_line.split()[2])
      w = float(single_line.split()[3])
      h = float(single_line.split()[4])

      crop_x1 = int((x - w/2) * image_width)
      crop_x2 = int((x + w/2) * image_width)
      crop_y1 = int((y - h/2) * image_height)
      crop_y2 = int((y + h/2) * image_height)


      cropped_image = loaded_image[crop_y1:crop_y2, crop_x1:crop_x2]
      cv2.imwrite(f'/content/drive/MyDrive/darknet/data/crop_data/{label}/{data_name}{ind}.png', cropped_image)
  else :
    continue

5-3 잘라낸 이미지들과 roboflow에서 가져온 신호등 데이터들을 기존 이미지에 합성

In [ ]:
import cv2
import os
import matplotlib.pyplot as plt
import random
import glob
from tqdm import tqdm

In [ ]:
%cd /content/drive/MyDrive/darknet/data/

In [ ]:
os.mkdir('composite_data')

In [ ]:
%cd /content/drive/MyDrive/darknet/data/resize

In [ ]:
crop_img_dict = {}
for i in tqdm(['0','1','2','4','5','7']) :
  path = '/content/drive/MyDrive/darknet/data/crop_data/' + i
  img_list = os.listdir(path)
  load_img = []
  for image in img_list :
    load_img.append(cv2.imread(path + f'/{image}'))
  crop_img_dict[i] = load_img

In [ ]:
file_name = []

for i in glob.glob('*.png') :
    file_name.append(i[:-4])

In [ ]:
crop_data_path = '/content/drive/MyDrive/darknet/data/crop_data/'
txt_data_path = '/content/drive/MyDrive/darknet/data/dataset/'
composite_data_path = '/content/drive/MyDrive/darknet/data/composite_data/'
for name in tqdm(file_name[:7500]) :
    img_name = name + ".png"
    txt_name = name + ".txt"
    original_img = cv2.imread(img_name)

    try : # .shape가 안된다면 넘어간다. (안하면 에러가 난다)
        test = original_img.shape
    except :
        continue

    # txt 파일에 있는 label 갯수 확인
    try :
        with open(txt_data_path + txt_name, 'r') as f :
            txt_data = f.read().split('\n')
            n_label = len(txt_data)
    except :
        txt_data = []
        n_label = 0

    # object가 6개 이하인 img에만 합성
    max_object = 6

    composite_label = (max_object - n_label)

    # 이미 object가 6개 이상인 것들은 파일 복사만 한다.
    if composite_label <= 0 :
        continue
        cv2.imwrite(composite_data_path + name + '_0.png', original_img)
        with open(composite_data_path + name + '_1.txt', 'w') as f :
            a = '\n'.join(txt_data)
            if a[0] == '\n' :
                a = a[1:]
            f.write(a)
        continue

    # composite_label 갯수만큼 합성할 라벨 추첨
    random_label = []
    for i in range(composite_label) :
        random_label.append(random.choice(['0','0','0','1','1','1','2','2','4','4','5','5','5','7'])) # 데이터 갯수에 따라 비율 조정

    # crop 이미지 합성 반복
    crop_txt_data = []
    for rand_number in random_label :

        cropped_img = random.choice(crop_img_dict[rand_number]) # crop img 가져오기

        # img 정보
        crop_x, crop_y = cropped_img.shape[1], cropped_img.shape[0]

        # 이미지가 긴 쪽이 90 보다 낮으면 90까지 늘린다
        if max(crop_x, crop_y) <= 90 :
            ratio = 90 / max(crop_x, crop_y)
            cropped_img = cv2.resize(cropped_img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

        # 신호등 이미지중에 짧은 쪽이 70보다 크면 70까지 줄인다
        if rand_number in ['0', '1'] and min(crop_x, crop_y) >= 70 :
            ratio = 70 / min(crop_x, crop_y)
            cropped_img = cv2.resize(cropped_img, dsize=(0, 0), fx=ratio, fy=ratio, interpolation=cv2.INTER_LINEAR)

        # crop img 정보<ctr
        crop_x, crop_y = cropped_img.shape[1], cropped_img.shape[0]
        x = random.randint(0, original_img.shape[1] - cropped_img.shape[1])
        y = random.randint(0, original_img.shape[0] - cropped_img.shape[0])


        # 이미지 합성
        original_img[y:y+crop_y, x:x+crop_x] = cropped_img

        # txt 파일을 위한 x,y,w,h, 구하기
        txt_x = (x + crop_x / 2) / original_img.shape[1]
        txt_y = (y + crop_y / 2) / original_img.shape[0]
        txt_w = crop_x / original_img.shape[1]
        txt_h = crop_y / original_img.shape[0]

        # bbox 데이터 모으기
        crop_txt_data.append(f'{rand_number} {txt_x} {txt_y} {txt_w} {txt_h}')

    # 이미지 저장 - 데이터 구분을 위해 파일명 끝에 _0을 추가한다
    cv2.imwrite(composite_data_path + name + '_1.png', original_img)

    # 처음에 가져온 txt_data에 txt 데이터 모아놓은 것 붙혀넣는다
    txt_data += crop_txt_data
    txt_data = '\n'.join(txt_data)
    if txt_data[0] == '\n' :
        txt_data = txt_data[1:]

    # composite 데이터 추가한 후 파일에 새로 저장
    with open(composite_data_path + name + '_1.txt', 'w') as f :
        f.write(txt_data)

5-4 YOLO v8을 합성할 수 있게끔 데이터 옮기기

In [ ]:
%cd /content/drive/MyDrive/darknet/data/label_fixed
from_dataset = '/content/drive/MyDrive/darknet/data/composite_data/'
to_dataset = '/content/drive/MyDrive/Colab Notebooks/dataset/'
data = glob.glob('*.png')
label = glob.glob('*.txt')

x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=1001)

for data_type ,path in zip([x_train], ['train/images/']) :
    for file_name in tqdm(data_type) :
        shutil.copy(from_dataset + file_name, to_dataset + path + file_name)

# yaml 파일은 직접 파일을 열어서 수정을 하였다

5-5 YOLO v8 학습

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

from ultralytics import YOLO

In [ ]:
# Load YOLOv8n, train it on COCO128 for 3 epochs and predict an image with it
model = YOLO('/content/drive/MyDrive/Colab Notebooks/yolo_yaml/yolov8.yaml')  # build a new model from scratch
model = YOLO('/content/drive/MyDrive/Colab Notebooks/yolo_pt/yolov8m.pt')  # load a pretrained YOLOv8M detection model
model.train(data='/content/drive/MyDrive/Colab Notebooks/dataset/data.yaml', epochs=5)  # train the model

In [ ]:
import shutil # 혹시 몰라서 runs 파일 통째로 드라이브로 옮겼다
shutil.move('/content/runs', '/content/drive/MyDrive')